In [1]:
import torch

from childnet import ChildNetwork
from cifar10_training import Cifar10Trainer
import controller_training
from controller import ControllerNetwork
from utils import layer_sampler, connection_sampler, exponential_moving_average

In [2]:
outer_iterations = 310 #310
controller_iterations = 5  #2000
child_iterations = 2 #2
##
hidden_size = 100
num_layer_types = 4
num_layers = 6
num_filters = 32
num_classes = 10  # cifar-10 dataset
grad_clip_value = 0.15
##
val_accuracy_list = []


train_time_list = []

In [3]:

cifar10_trainer = Cifar10Trainer(batch_size=128)

what is the device? cpu or gpu ?  cuda
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
controller = ControllerNetwork(hidden_size, num_layers, num_layer_types)
load_previous_network_weights = None
#
for i in range(outer_iterations):
    print("ITERATION NUMBER %d" % (i+1))
    ############## sample model ##############
    print('Controller is sampling architecture...')
    soft_1, soft_2 = controller()
    #
    print(soft_1)
    layers_types = layer_sampler(soft_1)
    # print(layers_types.shape)
    # take log after sampling (we couldn't do it before sampling)
    soft_1 = torch.log(soft_1)
    #
    skip_connections = connection_sampler(soft_2)
    ############## initialize weights ##############
    print('Creating new child network...')
    if i == 0:
        load_previous_network_weights = False
    child_net = ChildNetwork(layers_types, skip_connections, load_previous_network_weights,
                             num_hidden_layers=num_layers,
                             num_filters=num_filters, num_classes=num_classes)
    load_previous_network_weights = True

    ############## train model ##############
    print('Training child network...')
    # if(i > 0):
    #     child_net.load_state_dict(torch.load('weights.pth'))
    val_accuracy, new_train_time_list = cifar10_trainer.train(child_net, no_epoch=child_iterations)
    val_accuracy_list.append(val_accuracy)
    train_time_list.extend(new_train_time_list)
    ############## update weights ##############
    print('Saving child network\'s weights...')
    # torch.save(child_net.state_dict(), 'weights.pth')

    ############## train controller ##############
    print('Training controller network')
    baseline = exponential_moving_average(val_accuracy_list)
    multiplier = val_accuracy - baseline
    print('val_accuracy is = ', val_accuracy)
    print('baseline is = ', baseline)
    print('multiplier is = ', multiplier)
    controller_training.train(controller, soft_1, soft_2, layers_types, skip_connections, multiplier, grad_clip_value=grad_clip_value, no_epoch=controller_iterations)
    ##############################################
    print('##############################################')
    print('##############################################')

ITERATION NUMBER 1
Controller is sampling architecture...
tensor([[[0.2533, 0.2556, 0.2491, 0.2421]],

        [[0.2523, 0.2553, 0.2495, 0.2429]],

        [[0.2518, 0.2547, 0.2499, 0.2436]],

        [[0.2528, 0.2601, 0.2447, 0.2425]],

        [[0.2524, 0.2630, 0.2425, 0.2421]],

        [[0.2526, 0.2678, 0.2386, 0.2410]],

        [[0.2520, 0.2706, 0.2370, 0.2405]],

        [[0.2518, 0.2744, 0.2342, 0.2396]],

        [[0.2511, 0.2767, 0.2331, 0.2390]],

        [[0.2509, 0.2797, 0.2312, 0.2383]],

        [[0.2503, 0.2815, 0.2304, 0.2378]],

        [[0.2501, 0.2839, 0.2289, 0.2371]]], grad_fn=<CatBackward>)
Creating new child network...
Training child network...


In [ ]:
import matplotlib.pyplot as plt

plt.plot(val_accuracy_list)
plt.figure()
plt.plot(train_time_list)